In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import Compose, ToTensor, Resize, CenterCrop
from torchmetrics import AUROC
from torch.utils.data import DataLoader
from timm import models
from PIL import Image
import os

In [2]:
###Modify Me according to your paths###
ckpt_dir = "checkpoints/"
ood_data_dir = "data/OOD/"
id_data_dir = "data/ID/"

# prepare data

500 ID images and 500 OOD images
target for ID images is 1 and target for OOD images is 0. (binary OOD detection)

In [3]:
ori_preprocess = Compose([
        Resize((224), interpolation=Image.BICUBIC),
        CenterCrop(size=(224, 224)),
        ToTensor()])

/home/mamooler/anaconda3/envs/sem_proj/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [4]:
ood_images = []

for folder in os.listdir(ood_data_dir):
    for img_path in os.listdir(ood_data_dir+folder):
        ood_images.append(Image.open(ood_data_dir+folder+"/"+img_path))

ood_data = torch.zeros((len(ood_images), 3, 224, 224))
ood_targets = torch.zeros(len(ood_data), dtype=torch.int)
for ind, img in enumerate(ood_images):
    image_tensor = ori_preprocess(img)
    ood_data[ind] = image_tensor


id_images = []

for folder in os.listdir(id_data_dir):
    for img_path in os.listdir(id_data_dir+folder):
        id_images.append(Image.open(id_data_dir+folder+"/"+img_path))


id_data = torch.zeros((len(id_images), 3, 224, 224))
id_targets = torch.ones(len(id_data), dtype=torch.int)
for ind, img in enumerate(id_images):
    image_tensor = ori_preprocess(img)
    id_data[ind] = image_tensor


data = torch.cat((ood_data, id_data))
targets = torch.cat((ood_targets, id_targets))

# data = torchvision.datasets.CIFAR10(data_dir, download=True)
# data_loader = DataLoader((data, target), batch_size=4, shuffle=True)

In [5]:
eval_metric = AUROC(num_classes=1000)

/home/mamooler/anaconda3/envs/sem_proj/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


# ResNet

In [6]:
class ResNet(nn.Module):
    def __init__(self, model_ckpt):
        super(ResNet, self).__init__()
        resnet = models.resnet50()
        ckpt = torch.load(f"checkpoints/resnet50_a2_0-a2746f79.pth")
        resnet.load_state_dict(ckpt) 
        self.resnet = resnet
        self.softmax = nn.Softmax()
    
    def forward(self, image):
        res = self.resnet(image)
        res = self.softmax(res)
        
        return res
    

resnet = ResNet(f"resnet50_a2_0-a2746f79.pth")
resnet_preds = resnet(data)


/home/mamooler/anaconda3/envs/sem_proj/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /tmp/pip-req-build-1wfnpb1e/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/tmp/ipykernel_20225/2131796798.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = self.softmax(res)


In [7]:
resnet_auroc = eval_metric(resnet_preds, targets)
print(resnet_auroc)

/home/mamooler/anaconda3/envs/sem_proj/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


tensor(0.0008)


# DeiT

In [8]:
class DeiT(nn.Module):
    def __init__(self, model_ckpt):
        super(DeiT, self).__init__()
        deit =  torch.hub.load('facebookresearch/deit:main', 'deit_small_patch16_224')
        ckpt = torch.load(f"checkpoints/deit_small_patch16_a2_0-83b53863.pth")
        deit.load_state_dict(ckpt) 
        self.deit = deit
        self.softmax = nn.Softmax()
    
    def forward(self, image):
        res = self.deit(image)
        res = self.softmax(res)
        
        return res
    

deit = DeiT(f"deit_small_patch16_a2_0-83b53863.pth")
deit_preds = deit(data)


Using cache found in /home/mamooler/.cache/torch/hub/facebookresearch_deit_main


In [ ]:
deit_auroc = eval_metric(deit_preds, targets)
print(deit_auroc)

Error: Kernel is dead